In [1]:
import torch
import os
from h36m_dataset import H36M_Dataset
from extra_dataset import extra_Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

## Load datasets

In [3]:
dataset_h36m_train = H36M_Dataset('', 10, 10, 1, split=0)
dataset_h36m_test = H36M_Dataset('', 10, 10, 1, split=1)
dataset_extra = extra_Dataset('', 10, 10, 1)
#10 frames for training, 10 for testing in each 17 joints 3d poses
print(dataset_h36m[0].shape)
print(dataset_extra[0].shape)

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

Reading subject 11, action sitting, subaction 2
Reading subject 11, action sittingdown, subaction 1
Reading subject 11, action sittingdown, subaction 2
Reading subject 11, action takingphoto, subaction 1
Reading subject 11, action takingphoto, subaction 2
Reading subject 11, action waiting, subaction 1
Reading subject 11, action waiting, subaction 2
Reading subject 11, action walkingdog, subaction 1
Reading subject 11, action walkingdog, subaction 2
Reading subject 11, action walkingtogether, subaction 1
Reading subject 11, action walkingtogether, subaction 2
Reading  VisionLabSS23_3DPoses\2021-08-04-singlePerson_000.json
Reading  VisionLabSS23_3DPoses\2021-08-04-singlePerson_001.json
Reading  VisionLabSS23_3DPoses\2021-08-04-singlePerson_002.json
Reading  VisionLabSS23_3DPoses\2021-08-04-singlePerson_003.json
Reading  VisionLabSS23_3DPoses\2022-05-26_2persons_000.json
Reading  VisionLabSS23_3DPoses\2022-05-26_2persons_001.json
Reading  VisionLabSS23_3DPoses\2022-05-26_2persons_002.jso

In [4]:
train_loader = DataLoader(dataset_h36m_train, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(dataset_h36m_test, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
extra_loader = DataLoader(dataset_extra, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [11]:
data = next(iter(train_loader)) 
print(data.shape)

torch.Size([32, 20, 51])
